In [1]:
#!/usr/bin/env python
# coding: utf-8

# In[1]:


import json
import glob
import numpy as np
import pickle
from tqdm.auto import tqdm
from collections import defaultdict
import random


In [2]:

# In[3]:


# load in wikipedia titles

titles2id = defaultdict(list)
corpora = "tpr_for_reference/corpora/nq/psgs_w100.tsv"
first = True
for line in tqdm(open(corpora, "r"), total=21015325):
    if first:
        first = False
        continue
    split = line.split("\t")
    if len(split) == 3:
        i = int(split[0].strip())
        text = split[1].strip()
        title = split[2].strip()
        titles2id[title].append(i)
    else:
        print("ERROR ",line)



  0%|          | 0/21015325 [00:00<?, ?it/s]

In [3]:

# In[4]:


# map KILT ids to titles:

i2t = {}
with open("all_KILT_pages_20190801.txt", "r") as fin:
    for line in fin.readlines():
        split = line.split("\t")
        i = int(split[0].strip())
        title = split[1].strip()
        i2t[i] = title




In [4]:
# In[5]:


# map titles to topics:

title2topic_list = defaultdict(list)
for idx,file in enumerate(glob.glob("topics_KILT/*")):
#     print(file)
    topic = file.split('/')[-1].replace('.csv', '') #idx+1
    print(topic)
    with open(file, "r") as fin:
        first = True
        for line in fin.readlines():
            if first:
                first = False
                continue
            split = line.split(",")
            if len(split)==3:
                i = int(split[0].strip())
                title = i2t[i]
                score = float(split[2].strip())
                title2topic_list[title].append([topic,score])

title2topic = {}
for title, lista in title2topic_list.items():
    max_topic = None
    max_score = None
    for x in lista:
        topic,score = x
        if not max_score or score > max_score:
            max_score = score
            max_topic = topic
    title2topic[title] = max_topic







Culture.Internet_culture
Culture.Food_and_drink
Culture.Biography.Biography*
Culture.Linguistics
Culture.Biography.Women
Culture.Literature
Culture.Media.Entertainment
Culture.Media.Books
Culture.Media.Media*
Culture.Media.Films
Culture.Media.Radio
Culture.Media.Software
Culture.Media.Music
Culture.Media.Television
Culture.Media.Video_games
Culture.Performing_arts
Culture.Philosophy_and_religion
Culture.Sports
Culture.Visual_arts.Comics_and_Anime
Culture.Visual_arts.Architecture
Culture.Visual_arts.Fashion
Geography.Geographical
Culture.Visual_arts.Visual_arts*
Geography.Regions.Africa.Northern_Africa
Geography.Regions.Africa.Africa*
Geography.Regions.Africa.Eastern_Africa
Geography.Regions.Africa.Central_Africa
Geography.Regions.Americas.North_America
Geography.Regions.Africa.Western_Africa
Geography.Regions.Africa.Southern_Africa
Geography.Regions.Americas.Central_America
Geography.Regions.Asia.East_Asia
Geography.Regions.Asia.North_Asia
Geography.Regions.Asia.Asia*
Geography.Regions

In [5]:
title2topic

{'Super Mario Kart': 'Culture.Internet_culture',
 'Star Wars: X-Wing Alliance': 'Culture.Internet_culture',
 'Romance of the Three Kingdoms (video game series)': 'Culture.Media.Video_games',
 'Riven': 'Culture.Internet_culture',
 'Uplink (video game)': 'Culture.Internet_culture',
 'Eggman': 'Culture.Media.Media*',
 'Pitfall!': 'Culture.Internet_culture',
 'Sonic the Fighters': 'Culture.Internet_culture',
 '2004 in video gaming': 'Culture.Media.Video_games',
 'Track & Field (video game)': 'Culture.Internet_culture',
 'Pengo (video game)': 'Culture.Internet_culture',
 'Video game development': 'Culture.Internet_culture',
 'Lists of Nintendo characters': 'Culture.Media.Video_games',
 'Magic: The Gathering video games': 'Culture.Media.Video_games',
 'Tecmo': 'Culture.Media.Video_games',
 'Rampage (video game)': 'Culture.Media.Video_games',
 'Sheeva': 'Culture.Media.Video_games',
 'Geoff Crammond': 'Culture.Media.Video_games',
 'Virtua Racing': 'Culture.Internet_culture',
 'Sonic Labyrinth'

In [6]:
# In[9]:


# map passages to topics for export:

passage2topic = {}

first = True
for line in tqdm(open(corpora, "r"), total=21015325):
    if first:
        first = False
        continue
    split = line.split("\t")
    if len(split) == 3:
        i = int(split[0].strip())
        text = split[1].strip()
        title = split[2].strip()
        if title in title2topic:
            passage2topic[(title, text)] = title2topic[title]
    else:
        print("ERROR ",line)



  0%|          | 0/21015325 [00:00<?, ?it/s]

In [8]:
#passage2topic

In [ ]:
# In[10]:


# map questions to topics for export:

def load_data(filename):
    data = []
    with open(filename, "r") as fin:
        lines = fin.readlines()
        for line in lines:
            data.append(json.loads(line))
    return data

question2topic = {}
for (split, fi) in [
    ('train', 'nq-train-kilt.jsonl'),
    ('dev', 'nq-dev-kilt.jsonl'),
]:
    kilt_data = data/data(fi)
    q2t = {}
    for q in kilt_data:
        try:
            q2t[q["input"].strip()] = title2topic[q["output"][0]["provenance"][0]["title"].strip()]
        except:
            pass
    question2topic[split] = q2t



In [10]:
# In[13]:


# dump results:

# with open('question2topic.train.jsonl', 'w') as f:
#     for k, v in question2topic['train'].items():
#         f.write(json.dumps({'question': k, 'topic': v})+'\n')
    
# with open('question2topic.dev.jsonl', 'w') as f:
#     for k, v in question2topic['dev'].items():
#         f.write(json.dumps({'question': k, 'topic': v})+'\n')

with open('data/passage2topic.jsonl', 'w') as f:
    for (title, text), topic in tqdm(passage2topic.items()):
        f.write(json.dumps({'title': title, 'text':text, 'topic': topic})+'\n')


# In[ ]:


  0%|          | 0/1830410 [00:00<?, ?it/s]